<a href="https://colab.research.google.com/github/sovank/learn-linux/blob/main/Day_83_Advanced_Shell_Scripting_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q15. DevOps Class - 8 Assignment - 1

Raw Problem

Objective:

The goal is to write a script in /home/user/server_health_check.sh that monitors system processes and logs memory usage. The script will accomplish the following tasks:

TASK: Logging Process ID and Memory Usage

The script should continuously monitor every 2 sec the system processes and log the Process ID (PID) and memory usage for each process.
The information should be logged in the format PID_ID Memory in a file located at /home/user/health_reports/server_health.log.
This log file will help track the memory usage of various processes over time.

Expected Output:
A file named server_health.log located in /home/user/health_reports/, which contains entries in the format PID_ID Memory for each monitored process.

Sample Output:

1234 45MB

5678 30MB

9101 100MB

In [ ]:
#!/bin/bash

server_log_file="/home/user/health_reports/server_health.log"

monitor_memory_usage()
{
        > $server_log_file
        # empties the SERVER_HEALTH_LOG file (if it already exists) by redirecting an empty output to it.
        # This clears out any old data, ensuring that each time the function starts, it begins with a fresh log.

        while true
        do
                # ps is a command that shows information about active processes.
                # -e shows information about every running process.
                # -o pid,rss specifies the output format to include only the pid (process ID) and rss
                ps -eo pid,rss | awk 'NR>1 {print $1, $2}' | while read -r pid rss
                # NR>1 skips the first line (header) so only actual process information is processed.
                # rss (Resident Set Size, the non-swapped physical memory used by the process, in kilobytes).
                do
                    mem_mb=$((rss/1024))
                    echo "$pid ${mem_mb}MB" > $server_log_file
                done

                sleep 2
        done
}


monitor_memory_usage

\



---



# Q16. DevOps Class - 8 Assignment - 2

Objective:

Optimize the previous script to alert whenever certain memory limit exceeds.

Task: Alerting on Memory Usage Exceeding 20MB

The script should monitor the memory usage of each process and log any process that consumes more than 20MB of memory.
If a process exceeds 20MB of memory usage, the script should log the Process ID (PID) and memory usage in a file named /home/user/health_reports/alert.log.

Only processes that exceed the 20MB threshold should be logged in the alert file. Processes below this threshold should not appear in the alert log.

Expected Output:
A file named alert.log located in /home/user/health_reports/, which logs entries in the format PID_ID Memory for processes that exceed 20MB of memory usage.

Sample Output:

534 23MB

1129 58MB

In [ ]:
#!/bin/bash

server_log_file="/home/user/health_reports/server_health.log"
alert_log_file="/home/user/health_reports/alert.log"

monitor_memory_usage()
{
        > $server_log_file
        > $alert_log_file
        # empties the SERVER_HEALTH_LOG and alert_log_file (if it already exists) by redirecting an empty output to it.
        # This clears out any old data, ensuring that each time the function starts, it begins with a fresh log.

        while true
        do
                # ps is a command that shows information about active processes.
                # -e shows information about every running process.
                # -o pid,rss specifies the output format to include only the pid (process ID) and rss
                ps -eo pid,rss | awk 'NR>1 {print $1, $2}' | while read -r pid rss
                # NR>1 skips the first line (header) so only actual process information is processed.
                # rss (Resident Set Size, the non-swapped physical memory used by the process, in kilobytes).
                do
                        mem_mb=$((rss/1024))
                        echo "$pid ${mem_mb}MB" >> $server_log_file

                        if [ ${mem_mb} -gt 20 ]
                        then
                                echo "$pid ${mem_mb}MB" >> $alert_log_file
                        fi
                done

                sleep 2
        done
}

monitor_memory_usage # very important to call the function

\



---



# Q17. DevOps Class - 8 Assignment - **3**

Objective:

Extending the previous question, regular logs are being stored in the logfile.log file. Your task is to write a script in website_health_check.sh file that fulfills the below tasks:

TASK - 1:

Redirect only the errors from this log file to the error_reports/website_health.log file. Remember, error storing should be continuous and dynamic.

Tip: Create the report before running the test cases.

Output:

2024-07-22 23:11:15 2024-07-22 23:10:55 ERROR: This is error number 1.
2024-07-22 23:11:15 2024-07-22 23:11:00 ERROR: This is error number 2.
2024-07-22 23:11:15 2024-07-22 23:11:05 ERROR: This is error number 3.

In [ ]:
#!/bin/bash

LOG_FILE="/home/user/logfile.log"
ERROR_LOG="/home/user/error_reports/website_health.log"
REPORT_FILE="/home/user/error_reports/website_report.log"

capture_errors() {
    tail -n 0 -F "$LOG_FILE" | while read -r line; do
    # tail -n 0: Starts reading from the end of the file (no previous lines are shown).
    #-F: Continuously follows LOG_FILE and adjusts if the file is rotated (recreated or renamed).
        if [[ "$line" == *"ERROR"* ]]; then
            echo "$line" >> "$ERROR_LOG"
        fi
    done
}

capture_errors &



---



TASK - 2:

Based on the above errors received, prepare an error report and store the necessary details in the error_reports/website_report.log file.

Output Format:

Error Report - Mon Jan 2 20:10:20 UTC 2023
Total Errors: 9
Latest Error: 2023-01-02 20:10:16 2023-01-02 20:10:16 ERROR: This is error number 21.

In [ ]:
#!/bin/bash

LOG_FILE="/home/user/logfile.log"
ERROR_LOG="/home/user/error_reports/website_health.log"
REPORT_FILE="/home/user/error_reports/website_report.log"

capture_errors() {
    tail -n 0 -F "$LOG_FILE" | while read -r line; do
    # tail -n 0: Starts reading from the end of the file (no previous lines are shown).
    #-F: Continuously follows LOG_FILE and adjusts if the file is rotated (recreated or renamed).
        if [[ "$line" == *"ERROR"* ]]; then
            echo "$line" >> "$ERROR_LOG"
        fi
    done
}

generate_report() {
    while true; do
        sleep 10

        if [ ! -f "$ERROR_LOG" ] || [ ! -s "$ERROR_LOG" ]; then
        # -f "$ERROR_LOG": Checks if ERROR_LOG exists as a regular file.
        # -s "$ERROR_LOG": Checks if ERROR_LOG is non-empty.
            echo "No errors found to report."
            echo "Error Report - $(date)" > "$REPORT_FILE"
            echo "Total Errors: 0" >> "$REPORT_FILE"
            echo "Latest Error: None" >> "$REPORT_FILE"
            echo "Report generated at $REPORT_FILE"
            continue
        fi

        local error_count
        error_count=$(grep -c "ERROR" "$ERROR_LOG")
        local latest_error
        latest_error=$(tail -n 1 "$ERROR_LOG")

        echo "Error Report - $(date)" > "$REPORT_FILE"
        echo "Total Errors: $error_count" >> "$REPORT_FILE"
        echo "Latest Error: $latest_error" >> "$REPORT_FILE"

        echo "Report generated at $REPORT_FILE"
    done
}

capture_errors &

sleep 5

generate_report